In [1]:
from IPython.display import HTML
HTML('<a href="https://www.odoo.sh/" target="_blank">Get Odoo.sh Trial</a>')

In [2]:
import os
from google.colab import files

# Create module directory structure
os.makedirs('quickdesk/models', exist_ok=True)
os.makedirs('quickdesk/views', exist_ok=True)
os.makedirs('quickdesk/security', exist_ok=True)
os.makedirs('quickdesk/data', exist_ok=True)

# Create __init__.py files
with open('quickdesk/__init__.py', 'w') as f:
    f.write("from . import models")

with open('quickdesk/models/__init__.py', 'w') as f:
    f.write("from . import ticket, category, user")

# Create __manifest__.py
manifest_content = """{
    "name": "QuickDesk",
    "version": "1.0",
    "author": "Your Name",
    "summary": "Help Desk System",
    "description": "A comprehensive help desk solution for Odoo",
    "category": "Services/Helpdesk",
    "depends": ["base", "mail"],
    "data": [
        "security/ir.model.access.csv",
        "security/record_rules.xml",
        "views/ticket_views.xml",
        "views/category_views.xml",
        "views/user_views.xml",
        "data/mail_template.xml"
    ],
    "demo": [],
    "installable": True,
    "application": True,
    "auto_install": False
}"""

with open('quickdesk/__manifest__.py', 'w') as f:
    f.write(manifest_content)

# Create model files
ticket_model = """from odoo import models, fields, api

class Ticket(models.Model):
    _name = 'quickdesk.ticket'
    _description = 'Support Ticket'
    _inherit = ['mail.thread', 'mail.activity.mixin']

    name = fields.Char(string='Subject', required=True, tracking=True)
    description = fields.Html(string='Description')
    category_id = fields.Many2one('quickdesk.category', string='Category', tracking=True)
    user_id = fields.Many2one('res.users', string='Created By', default=lambda self: self.env.user, tracking=True)
    assigned_to = fields.Many2one('res.users', string='Assigned To', tracking=True)
    state = fields.Selection([
        ('open', 'Open'),
        ('in_progress', 'In Progress'),
        ('resolved', 'Resolved'),
        ('closed', 'Closed')],
        default='open',
        tracking=True,
        group_expand='_expand_states')
    attachment_ids = fields.Many2many('ir.attachment', string='Attachments')
    vote_count = fields.Integer(string='Votes', default=0)

    def _expand_states(self, states, domain, order):
        return [key for key, val in type(self).state.selection]

    def action_in_progress(self):
        self.write({'state': 'in_progress'})

    def action_resolve(self):
        self.write({'state': 'resolved'})

    def action_close(self):
        self.write({'state': 'closed'})

    def action_upvote(self):
        self.vote_count += 1

    def action_downvote(self):
        self.vote_count -= 1"""

with open('quickdesk/models/ticket.py', 'w') as f:
    f.write(ticket_model)

category_model = """from odoo import models, fields

class Category(models.Model):
    _name = 'quickdesk.category'
    _description = 'Ticket Category'

    name = fields.Char(string='Name', required=True)
    active = fields.Boolean(string='Active', default=True)"""

with open('quickdesk/models/category.py', 'w') as f:
    f.write(category_model)

user_model = """from odoo import models, fields

class ResUsers(models.Model):
    _inherit = 'res.users'

    quickdesk_ticket_ids = fields.One2many('quickdesk.ticket', 'user_id', string='Tickets')"""

with open('quickdesk/models/user.py', 'w') as f:
    f.write(user_model)

# Create security files
ir_access = """id,name,model_id:id,group_id:id,perm_read,perm_write,perm_create,perm_unlink
access_quickdesk_ticket_user,quickdesk.ticket.user,model_quickdesk_ticket,base.group_user,1,1,1,0
access_quickdesk_ticket_agent,quickdesk.ticket.agent,model_quickdesk_ticket,quickdesk.group_agent,1,1,1,1
access_quickdesk_ticket_admin,quickdesk.ticket.admin,model_quickdesk_ticket,quickdesk.group_admin,1,1,1,1
access_quickdesk_category_user,quickdesk.category.user,model_quickdesk_category,base.group_user,1,0,0,0
access_quickdesk_category_admin,quickdesk.category.admin,model_quickdesk_category,quickdesk.group_admin,1,1,1,1"""

with open('quickdesk/security/ir.model.access.csv', 'w') as f:
    f.write(ir_access)

record_rules = """<odoo>
    <record id="ticket_user_rule" model="ir.rule">
        <field name="name">User Tickets Only</field>
        <field name="model_id" ref="model_quickdesk_ticket"/>
        <field name="domain_force">['|',('user_id','=',user.id),('assigned_to','=',user.id)]</field>
        <field name="groups" eval="[(4, ref('base.group_user'))]"/>
    </record>
    <record id="ticket_agent_rule" model="ir.rule">
        <field name="name">Agent Access All Tickets</field>
        <field name="model_id" ref="model_quickdesk_ticket"/>
        <field name="groups" eval="[(4, ref('quickdesk.group_agent'))]"/>
    </record>
</odoo>"""

with open('quickdesk/security/record_rules.xml', 'w') as f:
    f.write(record_rules)

# Create view files
ticket_views = """<odoo>
    <!-- Ticket List View -->
    <record id="view_ticket_tree" model="ir.ui.view">
        <field name="name">ticket.tree</field>
        <field name="model">quickdesk.ticket</field>
        <field name="arch" type="xml">
            <tree>
                <field name="name"/>
                <field name="user_id"/>
                <field name="category_id"/>
                <field name="state"/>
                <field name="vote_count"/>
            </tree>
        </field>
    </record>

    <!-- Ticket Form View -->
    <record id="view_ticket_form" model="ir.ui.view">
        <field name="name">ticket.form</field>
        <field name="model">quickdesk.ticket</field>
        <field name="arch" type="xml">
            <form>
                <header>
                    <button name="action_in_progress" string="Start Progress" states="open" class="oe_highlight" type="object"/>
                    <button name="action_resolve" string="Resolve" states="in_progress" class="oe_highlight" type="object"/>
                    <button name="action_close" string="Close" states="resolved" class="oe_highlight" type="object"/>
                    <button name="action_upvote" string="Upvote" class="oe_highlight" type="object"/>
                    <button name="action_downvote" string="Downvote" type="object"/>
                    <field name="state" widget="statusbar"/>
                </header>
                <sheet>
                    <group>
                        <field name="name"/>
                        <field name="category_id"/>
                        <field name="assigned_to"/>
                        <field name="vote_count"/>
                    </group>
                    <group>
                        <field name="description"/>
                    </group>
                    <notebook>
                        <page string="Attachments">
                            <field name="attachment_ids"/>
                        </page>
                        <page string="Discussion">
                            <field name="message_ids" mode="tree"/>
                        </page>
                    </notebook>
                </sheet>
            </form>
        </field>
    </record>

    <!-- Action and Menu Items -->
    <record id="action_ticket" model="ir.actions.act_window">
        <field name="name">Tickets</field>
        <field name="res_model">quickdesk.ticket</field>
        <field name="view_mode">tree,form</field>
    </record>

    <menuitem id="menu_quickdesk_root" name="QuickDesk"/>
    <menuitem id="menu_quickdesk_tickets" name="Tickets" parent="menu_quickdesk_root" action="action_ticket"/>
</odoo>"""

with open('quickdesk/views/ticket_views.xml', 'w') as f:
    f.write(ticket_views)

category_views = """<odoo>
    <record id="view_category_tree" model="ir.ui.view">
        <field name="name">category.tree</field>
        <field name="model">quickdesk.category</field>
        <field name="arch" type="xml">
            <tree>
                <field name="name"/>
                <field name="active"/>
            </tree>
        </field>
    </record>

    <record id="view_category_form" model="ir.ui.view">
        <field name="name">category.form</field>
        <field name="model">quickdesk.category</field>
        <field name="arch" type="xml">
            <form>
                <sheet>
                    <group>
                        <field name="name"/>
                        <field name="active"/>
                    </group>
                </sheet>
            </form>
        </field>
    </record>

    <record id="action_category" model="ir.actions.act_window">
        <field name="name">Categories</field>
        <field name="res_model">quickdesk.category</field>
        <field name="view_mode">tree,form</field>
    </record>

    <menuitem id="menu_quickdesk_categories" name="Categories" parent="menu_quickdesk_root" action="action_category" sequence="10"/>
</odoo>"""

with open('quickdesk/views/category_views.xml', 'w') as f:
    f.write(category_views)

user_views = """<odoo>
    <record model="ir.ui.view" id="view_users_form_quickdesk">
        <field name="name">res.users.form.quickdesk</field>
        <field name="model">res.users</field>
        <field name="inherit_id" ref="base.view_users_form"/>
        <field name="arch" type="xml">
            <notebook position="inside">
                <page string="QuickDesk">
                    <field name="quickdesk_ticket_ids"/>
                </page>
            </notebook>
        </field>
    </record>
</odoo>"""

with open('quickdesk/views/user_views.xml', 'w') as f:
    f.write(user_views)

# Create email template
mail_template = """<odoo>
    <record id="ticket_created_email" model="mail.template">
        <field name="name">Ticket Created</field>
        <field name="model_id" ref="model_quickdesk_ticket"/>
        <field name="subject">New Ticket Created: ${object.name}</field>
        <field name="email_from">${user.email}</field>
        <field name="email_to">${object.assigned_to.email or ''}</field>
        <field name="body_html">
            <div>
                <p>Hello,</p>
                <p>A new ticket has been created:</p>
                <p>Subject: ${object.name}</p>
                <p>Status: ${object.state}</p>
                <p>You can view the ticket here: ${object.get_base_url()}/web#id=${object.id}&model=quickdesk.ticket</p>
            </div>
        </field>
    </record>
</odoo>"""

with open('quickdesk/data/mail_template.xml', 'w') as f:
    f.write(mail_template)

# Zip the module
!zip -r quickdesk.zip quickdesk

# Download the module
files.download('quickdesk.zip')

print("QuickDesk module created and downloaded as quickdesk.zip")

  adding: quickdesk/ (stored 0%)
  adding: quickdesk/views/ (stored 0%)
  adding: quickdesk/views/ticket_views.xml (deflated 77%)
  adding: quickdesk/views/category_views.xml (deflated 75%)
  adding: quickdesk/views/user_views.xml (deflated 57%)
  adding: quickdesk/data/ (stored 0%)
  adding: quickdesk/data/mail_template.xml (deflated 60%)
  adding: quickdesk/security/ (stored 0%)
  adding: quickdesk/security/ir.model.access.csv (deflated 72%)
  adding: quickdesk/security/record_rules.xml (deflated 62%)
  adding: quickdesk/__init__.py (stored 0%)
  adding: quickdesk/models/ (stored 0%)
  adding: quickdesk/models/user.py (deflated 27%)
  adding: quickdesk/models/category.py (deflated 36%)
  adding: quickdesk/models/__init__.py (stored 0%)
  adding: quickdesk/models/ticket.py (deflated 62%)
  adding: quickdesk/__manifest__.py (deflated 50%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

QuickDesk module created and downloaded as quickdesk.zip
